# OCR

> Use Azure Cognite Service Text Analytics to build an Optical Character Recognition program!

## Setup

Import Azure specific modules and load Azure Cognitive Services configuration from environment variables:

In [ ]:
import io
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from typing import Any, Tuple
from dotenv import load_dotenv
from PIL import Image

# Computer Vision modules
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials


load_dotenv()

cog_endpoint = os.getenv("COG_SERVICE_ENDPOINT")
cog_key = os.getenv("COG_SERVICE_KEY")

## Optical Character Recognition

Implement the following function given an image file path, returning the detected text language and all the detected texts data (text, rectangle coordinates):

In [ ]:
def get_texts(image_data: bytes) -> Tuple[str, Any]:
    # Instantiate Cognitive Services credentials using `cog_key`
    credentials = CognitiveServicesCredentials(cog_key)

    # Instantiate Computer Vision Client using `cog_endpoint` and `credentials` variables
    client = ComputerVisionClient(cog_endpoint, credentials)

    # Use a binary stream generator from the image data
    image_stream = io.BytesIO(image_data)
    
    # Analyze the image stream to extract all printed texts
    results = client.recognize_printed_text_in_stream(image_stream)

    # Extract the detected text language
    language = results.language
    
    # Extract all the detected texts data (position, text)
    texts = [
        dict(
            position=tuple(map(int, r.bounding_box.split(","))),
            text=" ".join([w.text for l in r.lines for w in l.words]),
        )
        for r in results.regions
    ]

    return language, texts

Test your OCR function:

In [ ]:
def annotate_image(image_data: bytes, language: str, texts: Any) -> None:
    print("Detected text language:", language)

    fig, ax = plt.subplots()
    ax.imshow(Image.open(io.BytesIO(image_data)))

    for t in texts:
        print(t["position"], t["text"])

        box = patches.Rectangle(
            (t["position"][0], t["position"][1]),
            t["position"][2],
            t["position"][3],
            linewidth=1, edgecolor="r", facecolor="none",
        )
        ax.add_patch(box)
        ax.annotate(
            t["text"],
            (t["position"][0], t["position"][1]),
            color="b",
        )


def handle_upload(data):
    image_data = data["owner"].data[0]
    language, texts = get_texts(image_data)
    annotate_image(image_data, language, texts)


photo_upload = widgets.FileUpload(
    accept="image/*",
    multiple=False,
)
photo_upload.observe(handle_upload, names="_counter")
photo_upload

## Solution

To see a potential solution, execute the following cell:

In [ ]:
%load ocr_solution.py